In [1]:
import os
from dotenv import load_dotenv
from phi.agent import Agent
from phi.tools.slack import SlackTools
from phi.model.groq import Groq
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

import requests

In [2]:
load_dotenv()
SLACK_TOKEN=os.getenv('SLACK_TOKEN')
GROQ_API_KEY=os.getenv('GROQ_API_KEY')

In [3]:
slack_agent = Agent(
    name="Cool Slack Agent",
    model=Groq(id = 'llama-3.3-70b-versatile',
               api_key=GROQ_API_KEY),
    tools=[SlackTools()],
    instructions=["Always include sources"],
    show_tool_calls=True,
    markdown=True,
)

In [4]:
# Function to fetch a random tech fact (general facts API for now)
def get_tech_fact():
    url = "https://uselessfacts.jsph.pl/random.json?language=en"
    response = requests.get(url)
    data = response.json()
    
    if not data:
        return "Could not fetch a fact, please try again later."
    
    fact = data['text']
    
    # Return tech fact as a simple string
    return f"Tech Fact: {fact}"

In [5]:
# Slack bot function to send messages
def send_slack_message(message, slack_token, channel="#tech-progress"):
    client = WebClient(token=slack_token)
    
    try:
        response = client.chat_postMessage(
            channel=channel,
            text=message
        )
        return response
    except SlackApiError as e:
        return f"Error sending message to Slack: {e.response['error']}"


In [6]:
# Groq agent function to fetch tech fact and send to Slack
def tech_fact_to_slack():
    fact_message = get_tech_fact()
    
    # Send message to Slack
    send_slack_message(fact_message, SLACK_TOKEN)

    # Return the fact message as a string
    return fact_message


In [7]:
# Create the Groq Agent with the tech fact tool
groq_agent = Agent(
    name="Tech Fact Bot",
    model=Groq(id = 'llama-3.3-70b-versatile',
               api_key=GROQ_API_KEY),
    tools=[tech_fact_to_slack],  # Add tech_fact_to_slack as a tool
    instructions=["Always include sources"],
    show_tool_calls=True,
    markdown=True,
    description="Fetches random tech facts and sends them to Slack."
)

In [8]:

# Trigger the agent to send tech fact to Slack
groq_agent.print_response({"role": "user", "content": "Send the tech fact to Slack."}, markdown=True)


Output()

KeyboardInterrupt: 